# 🚀 프로젝트 개요
이번에는 WandB를 사용해서 precision recall과 ROC curves 그리고 confusion matricse 기록하는 실습을 할 예정입니다.  
아이리스 데이터를 이용할 예정이고 사용 모델로는 sklearn의 Naive Bayes 분류기인 CategoricalNB을 사용합니다.

### 0️⃣ 필요 모듈 설치 후 import

In [ ]:
!pip install --upgrade wandb -qq

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import GaussianNB, CategoricalNB

import wandb
wandb.init(project="vega-plots")

### 1️⃣ 데이터 준비 후 모델 학습

In [ ]:
# 데이터 준비
iris = load_iris()
iris_data = iris.data
iris_label = iris.target

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data, 
                                                    iris_label, 
                                                    test_size=0.3, 
                                                    random_state=7)

# 모델 학습
cnb  = CategoricalNB()
cnb .fit(X_train, y_train)

# 데이터 예측
y_pred = cnb .predict(X_test)
y_prob_pred = cnb .predict_proba(X_test)

#roc auc score 추출
roc_auc_score(y_test, y_prob_pred, multi_class='ovo', average='weighted')

### 2️⃣ ROC curve 시각화 해보기

In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_prob_pred[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);  

### 3️⃣ Weights & Biases를 이용해서 그래프 기록하기

In [ ]:
# ROC
wandb.log({'roc': wandb.plots.ROC(y_test, y_prob_pred, cnb.classes_)})

# Precision Recall
wandb.log({'pr': wandb.plots.precision_recall(y_test, y_prob_pred, cnb.classes_)})

# Learning Curve
wandb.sklearn.plot_learning_curve(cnb, X_test, y_test)

# Confusion Matrix
wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels=cnb.classes_)

### 4️⃣ Weights & Biases를 이용해서 히트맵 기록하기

In [ ]:
wandb.init(project="vega-plots", name="Heatmap")

matrix_values = np.random.rand(3, 3)
x_labels=['seto', 'vers', 'virg']
y_labels=['SETO', 'VERS', 'VIRG']
wandb.log({'heatmap_with_text': wandb.plots.HeatMap(x_labels, y_labels, matrix_values, show_text=True)})
wandb.log({'heatmap_no_text': wandb.plots.HeatMap(x_labels, y_labels, matrix_values, show_text=False)})
